In [328]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization,Conv3D,Dropout, MaxPooling3D, Flatten, Dense,Concatenate, Reshape, TimeDistributed, Input, concatenate, LSTM,  Conv2D, MaxPooling2D, RepeatVector
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt

In [10]:
# 일별로 묶여있는 서울 미세먼지데이터와 국내 기상데이터 불러오기
seoul_pm = pd.read_csv("./daily_seoulpm.csv", index_col = 0) 
climate_data = pd.read_csv("./daily_climate.csv", index_col = 0)

In [11]:
seoul_pm.head()

,"('PM10', '서울 서대문구')"
Unnamed: 0,
2021-01-01,20.6125
2021-01-02,22.7125
2021-01-03,30.7500
2021-01-04,27.0750
2021-01-05,22.1250


In [14]:
# 인덱스를 날짜형식으로 변경
seoul_pm.index = pd.to_datetime(seoul_pm.index)
climate_data.index = pd.to_datetime(climate_data.index)

In [16]:
seoul_pm.head()

,"('PM10', '서울 서대문구')"
Unnamed: 0,
2021-01-01,20.6125
2021-01-02,22.7125
2021-01-03,30.7500
2021-01-04,27.0750
2021-01-05,22.1250


In [17]:
climate_data.head()

,"('강수량', '90')","('강수량', '101')","('강수량', '102')","('강수량', '105')","('강수량', '108')","('강수량', '119')","('강수량', '127')","('강수량', '133')","('강수량', '140')","('강수량', '152')",...,month_11,month_12,hour_0,hour_3,hour_6,hour_9,hour_12,hour_15,hour_18,hour_21
2021-01-01,0.0,0.000,0.0250,0.0,0.0,0.0,0.0000,0.075,0.025,0.0,...,0.0,0.0,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
2021-01-02,0.0,0.000,0.0000,0.0,0.0,0.0,0.0000,0.000,0.000,0.0,...,0.0,0.0,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
2021-01-03,0.0,0.000,0.0000,0.0,0.0,0.0,0.0000,0.000,0.000,0.0,...,0.0,0.0,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
2021-01-04,0.0,0.000,0.1250,0.0,0.0,0.0,0.0000,0.000,0.000,0.0,...,0.0,0.0,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
2021-01-05,0.0,0.025,0.2125,0.0,0.0,0.0,0.0125,0.000,0.000,0.0,...,0.0,0.0,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125


In [35]:
# 에어로졸 이미지가 저장되어있는 데이터프레임 불러오기
img_df = pd.read_csv(r"C:\Windows\System32\01.Final_project\jayden\final_aerosol_df.csv", index_col = 0)

In [37]:
# 인덱스 날짜 형식으로 변환
img_df.index = pd.to_datetime(img_df.index)

In [38]:
img_df.head()

,File
DateTime,
2021-01-01 00:45:00,C:/Users/user/Desktop/koreaIT/aerosol_image/ne...
2021-01-01 01:45:00,C:/Users/user/Desktop/koreaIT/aerosol_image/ne...
2021-01-01 02:45:00,C:/Users/user/Desktop/koreaIT/aerosol_image/ne...
2021-01-02 00:45:00,C:/Users/user/Desktop/koreaIT/aerosol_image/ne...
2021-01-02 01:45:00,C:/Users/user/Desktop/koreaIT/aerosol_image/ne...


In [40]:
# 기상데이터와 동일하게 2024-05-19 까지만 추출
img_df = img_df.loc[:"2024-05-19 02:45:00",:]

In [41]:
# 서울미세먼지 결측치인 날짜들. 이미지에서도 삭제해야함
indices_to_remove = ["2021-01-22 00:45:00",
                     "2021-01-22 01:45:00",
                     "2021-01-22 02:45:00",
                     "2023-02-03 00:45:00",
                     "2023-02-03 01:45:00",
                     "2023-02-03 02:45:00",
                     "2023-02-04 00:45:00",
                     "2023-02-04 01:45:00",
                     "2023-02-04 02:45:00",
                     "2023-02-05 00:45:00",
                     "2023-02-05 01:45:00",
                     "2023-02-05 02:45:00"]

In [42]:
# 위 날짜들을 삭제한 img_df를 img_df2에 저장
img_df2 = img_df.drop(indices_to_remove)

In [44]:
# 데이터 shape 확인
img_df2.shape

(3687, 1)

In [43]:
# 3장의 이미지가 보간기법으로 생성되어서 이미 전처리가되어 있음. 해당 이미지들은 아래 전처리 함수에서 제외해야하기때문에 확인.
# 보간적용한 사진 몇 번째 인덱스인지 확인하기
print(img_df2.index.get_loc("2021-01-26 00:45:00"))
# 보간적용한 사진 몇 번째 인덱스인지 확인하기
print(img_df2.index.get_loc("2021-10-08 02:45:00"))
# 보간적용한 사진 몇 번째 인덱스인지 확인하기
print(img_df2.index.get_loc("2022-02-15 00:45:00"))

72
836
1224


In [47]:
# date 인덱스 추출해두기
idx = seoul_pm.index

In [48]:
# 에어로졸 이미지 불러오기 및 전처리 함수
def preprocess_image(img_df2, no_crop_dates_indice, base_path): # 이미지데이터프레임, 전처리안할 이미지 인덱스번호, 이미지 경로 설정
    img_vectors = {
        '0045': [], # 00시 45분 이미지 벡터를 담을 리스트
        '0145': [], # 01시 45분 이미지 벡터를 담을 리스트
        '0245': [] # 02시 45분 이미지 벡터를 담을 리스트
    }

    for i in range(len(img_df2)):
        date = img_df2.index[i]
        img_filename = img_df2.iloc[i, 0][56:]  # 사용한 컴퓨터가 여러대여서 경로가 다 다르기때문에, 이미지파일명만 추출.
        img_path = os.path.join(base_path, img_filename) # 함수 호출 할때, 본인 컴퓨터내 에어로졸 이미지 경로는 base_path 자리에 설정.
        img = cv2.imread(img_path) # open cv 사용

        if img is not None:  # 이미지 로드가 성공했는지 확인
            if i not in no_crop_dates_indice:  # 인덱스가 no_crop_dates_indice에 있는지 확인
                img = img[150:1100, :]  # 위 아래 필요없는 정보 자르기
                img = img[:, 200:-200]  # 왼쪽 오른쪽 필요없는 정보 자르기
                img = img[400:1000, :]  # 추가 잘라내기 (필요시)

                # 이미지가 비어 있지 않은지 확인
                if img.size == 0: 
                    print(f"현재 {img_path} 이미지는 비어있습니다") # 만약 위의 크롭을 거치고 사이즈가 0이된 이미지가있는지 확인
                    continue # 0이 된게 있다면 스킵하기

            img = cv2.resize(img, (128, 128))  # 이미지 크기를 일관되게 조정
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # BGR을 RGB로 변환
            img = img / 255.0  # 정규화 (0~1 범위로 변경하기)

            time_key = date.strftime('%H%M') # date (인덱스) 에서 시간과 분만 추출
            if time_key in img_vectors:
                img_vectors[time_key].append(img) # 각 이미지가 00:45인지 01:45인지 02:45인지 확인 후, 알맞는 리스트로 append

    return img_vectors


# 크기 조정 제외할 이미지들의 인덱스 (날짜) 이미 보간법 적용된 사진들.
no_crop_dates_indice = [72, 836, 1224] # 72번, 836번, 1224번 사진은 crop 전처리 거치지않기.

# 이미지 파일 경로
base_path = "C:/Users/user/Desktop/koreaIT/aerosol_image/new_images2/"  # 현재 컴퓨터내 이미지폴더 디렉토리

# 이미지 벡터화
img_vectors = preprocess_image(img_df2, no_crop_dates_indice, base_path) # 함수 호출

# 데이터프레임 생성
result_df = pd.DataFrame({
    "index" : idx, # 추출해둔 인덱스로 date 생성
    '0045': img_vectors['0045'],
    '0145': img_vectors['0145'],
    '0245': img_vectors['0245']
})

In [49]:
result_df.shape

(1229, 4)

In [50]:
# 잘 만들어짐
result_df.head()

,index,0045,0145,0245
0,2021-01-01,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
1,2021-01-02,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
2,2021-01-03,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
3,2021-01-04,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
4,2021-01-05,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."


In [51]:
# 인덱스를 datetime 형식으로 변환
result_df = result_df.set_index("index")
result_df.index = pd.to_datetime(result_df.index)

In [117]:
result_df.head()

,0045,0145,0245
index,,,
2021-01-01,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
2021-01-02,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
2021-01-03,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
2021-01-04,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
2021-01-05,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...","[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."


In [118]:
result_df.shape

(1229, 3)

## 슬라이딩 윈도우 함수 생성 (시퀀스 함수 생성)

In [280]:
# 기상, 미세먼지 슬라이딩 윈도우 생성 함수
def create_sliding_window(climate_data, pm_data, past_days=6, future_days=3): # 기상데이터, 미세먼지데이터, 예측에 쓸 날짜수, 예측할 날짜수
    x_climate, y = [], []
    for i in range(len(climate_data) - past_days - future_days + 1): # 마지막 시퀀스까지 사용하기위해 마지막에 +1을 붙임
        # 과거 'past_days' 일의 기상 데이터
        climate_window = climate_data[i:i+past_days]
        # 미래 'future_days' 일의 미세먼지 데이터
        pm_window = pm_data[i+past_days:i+past_days+future_days]

        # 각 행에 결측치가 없는 경우에만 추가. 없다면 스킵.
        if not (pd.DataFrame(climate_window).isna().any().any() or pd.DataFrame(pm_window).isna().any().any()):
            x_climate.append(climate_window)
            y.append(pm_window)

    return np.array(x_climate), np.array(y)

## 이미지 데이터 시퀀스 함수 생성

In [281]:
# 이미지 시퀀스 생성 함수. 세 개의 이미지 시퀀스 배열을 생성.
def img_sequence_data(df, n_past_days=6, n_future_days=3):
    x_aod_0045, x_aod_0145, x_aod_0245 = [], [], []
    
    for i in range(len(df) - n_past_days - n_future_days + 1):
        if df.iloc[i:i + n_past_days + n_future_days].isnull().values.any():
            continue  # 결측값이 있는 행은 스킵
        x_aod_0045.append(np.stack(df['0045'].iloc[i:i + n_past_days].values)) # 3차원배열로 변경
        x_aod_0145.append(np.stack(df['0145'].iloc[i:i + n_past_days].values))
        x_aod_0245.append(np.stack(df['0245'].iloc[i:i + n_past_days].values))   
    return np.array(x_aod_0045), np.array(x_aod_0145), np.array(x_aod_0245)

In [282]:
# 기상데이터, 미세먼지데이터, 에어로졸 이미지데이터를 한번에 시퀀스 데이터로 변환해주는 함수
def create_combined_sliding_window(climate_data, pm_data, aod_df, past_days=3, future_days=3):
    x_climate, x_aod_0045, x_aod_0145, x_aod_0245, y = [], [], [], [], []
    
    for i in range(len(climate_data) - past_days - future_days + 1):
        # 과거 'past_days' 일의 기상 데이터
        climate_window = climate_data[i:i+past_days]
        # 미래 'future_days' 일의 미세먼지 데이터
        pm_window = pm_data[i+past_days:i+past_days+future_days]
        
        # 에어로졸 이미지의 슬라이딩 윈도우
        aod_window_0045 = aod_df['0045'].iloc[i:i + past_days]
        aod_window_0145 = aod_df['0145'].iloc[i:i + past_days]
        aod_window_0245 = aod_df['0245'].iloc[i:i + past_days]

        # 결측치가 없는 경우에만 추가
        if not (pd.DataFrame(climate_window).isna().any().any() or
                pd.DataFrame(pm_window).isna().any().any() or
                aod_window_0045.isnull().any() or
                aod_window_0145.isnull().any() or
                aod_window_0245.isnull().any()):
            
            x_climate.append(climate_window)
            x_aod_0045.append(np.stack(aod_window_0045.values))
            x_aod_0145.append(np.stack(aod_window_0145.values))
            x_aod_0245.append(np.stack(aod_window_0245.values))
            y.append(pm_window)

    return (np.array(x_climate),
            np.array(x_aod_0045),
            np.array(x_aod_0145),
            np.array(x_aod_0245),
            np.array(y))

In [283]:
# 데이터 준비
x_climate, x_0045, x_0145, x_0245, y_pm = create_combined_sliding_window(climate_data, seoul_pm, result_df, past_days=6, future_days=3)

In [284]:
x_climate.shape

(1204, 6, 2220)

In [285]:
x_0045.shape

(1204, 6, 128, 128, 3)

In [286]:
x_0145.shape

(1204, 6, 128, 128, 3)

In [287]:
x_0245.shape

(1204, 6, 128, 128, 3)

In [288]:
y_pm.shape

(1204, 3, 1)

In [289]:
# 데이터 분할 (훈련, 검증, 테스트셋)
x_climate_train, x_climate_test,  x_0045_train,  x_0045_test, x_0145_train, x_0145_test, x_0245_train, x_0245_test, y_train, y_test = train_test_split(x_climate, x_0045, x_0145, x_0245, y_pm, test_size=0.2, random_state=42)
x_climate_train, x_climate_val, x_0045_train,  x_0045_val, x_0145_train, x_0145_val, x_0245_train, x_0245_val, y_train, y_val = train_test_split(x_climate_train, x_0045_train, x_0145_train, x_0245_train, y_train, test_size=0.2, random_state=42)

print(f'train shapes: {x_climate_train.shape}, {x_0045_train.shape},{x_0145_train.shape},{x_0245_train.shape}, {y_train.shape}')
print(f'validation shapes: {x_climate_val.shape}, {x_0045_val.shape},{x_0145_val.shape},{x_0245_val.shape}, {y_val.shape}')
print(f'test shapes: {x_climate_test.shape}, {x_0045_test.shape},{x_0145_test.shape},{x_0245_test.shape}, {y_test.shape}')

train shapes: (770, 6, 2220), (770, 6, 128, 128, 3),(770, 6, 128, 128, 3),(770, 6, 128, 128, 3), (770, 3, 1)
validation shapes: (193, 6, 2220), (193, 6, 128, 128, 3),(193, 6, 128, 128, 3),(193, 6, 128, 128, 3), (193, 3, 1)
test shapes: (241, 6, 2220), (241, 6, 128, 128, 3),(241, 6, 128, 128, 3),(241, 6, 128, 128, 3), (241, 3, 1)


In [290]:
# 기상 데이터 스케일링
ss = StandardScaler()

In [291]:
# 3차원을 스케일링할 수 없기 때문에, 2차원으로 변환 후 다시 3차원으로 재변환
climate_train_reshaped = x_climate_train.reshape(x_climate_train.shape[0], -1)
climate_val_reshaped = x_climate_val.reshape(x_climate_val.shape[0], -1)
climate_test_reshaped = x_climate_test.reshape(x_climate_test.shape[0], -1)

climate_train_scaled_reshaped = ss.fit_transform(climate_train_reshaped)
climate_val_scaled_reshaped = ss.transform(climate_val_reshaped)
climate_test_scaled_reshaped = ss.transform(climate_test_reshaped)

scaled_climate_train = climate_train_scaled_reshaped.reshape(x_climate_train.shape)
scaled_climate_val = climate_val_scaled_reshaped.reshape(x_climate_val.shape)
scaled_climate_test = climate_test_scaled_reshaped.reshape(x_climate_test.shape)

In [296]:
# 3D CNN 정의
input_aerosol_0045 = Input(shape=(6, 128, 128, 3)) # 과거 6일치 데이터, 이미지의 높이 x 너비 , 미래 3일 예측
input_aerosol_0145 = Input(shape=(6, 128, 128, 3))
input_aerosol_0245 = Input(shape=(6, 128, 128, 3))

def create_cnn(input_layer):
    x = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling3D((1, 2, 2))(x) # 2, 2, 2 넣었더니 에러떠서 공간적 특징을 줄여서 1, 2 ,2로 설정.
    x = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D((1, 2, 2))(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D((1, 2, 2))(x)
    x = Flatten()(x)
    return x

cnn_0045 = create_cnn(input_aerosol_0045)
cnn_0145 = create_cnn(input_aerosol_0145)
cnn_0245 = create_cnn(input_aerosol_0245)

# 기상 데이터 입력
input_weather = Input(shape=(6, scaled_climate_train.shape[2])) # 3, 2200이 들어감
weather_lstm = LSTM(128, activation='relu', return_sequences=False)(input_weather)

# 결합
combined = concatenate([cnn_0045, cnn_0145, cnn_0245, weather_lstm])
x = Dense(256, activation='relu')(combined)
x = Dense(128, activation='relu')(x)
output = Dense(3, activation='linear')(x)  # 미래 3일 예측

In [297]:
# 모델 정의
model = Model(inputs=[input_aerosol_0045, input_aerosol_0145, input_aerosol_0245, input_weather], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("./best_3D_CNN_model_2.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

In [298]:
# 모델확인
model.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_30 (InputLayer)       [(None, 6, 128, 128, 3)]     0         []                            
                                                                                                  
 input_31 (InputLayer)       [(None, 6, 128, 128, 3)]     0         []                            
                                                                                                  
 input_32 (InputLayer)       [(None, 6, 128, 128, 3)]     0         []                            
                                                                                                  
 conv3d_55 (Conv3D)          (None, 6, 128, 128, 32)      2624      ['input_30[0][0]']            
                                                                                           

In [299]:
# 모델 훈련
history = model.fit([x_0045_train, x_0145_train, x_0245_train, scaled_climate_train], y_train, epochs=20, batch_size=32,
                    validation_data=([x_0045_val, x_0145_val, x_0245_val, scaled_climate_val], y_val), callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/20
25/25 [==============================] - ETA: 0s - loss: 1206.8075 

C:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


25/25 [==============================] - 281s 11s/step - loss: 1206.8075 - val_loss: 1470.0251
Epoch 2/20
25/25 [==============================] - 262s 10s/step - loss: 494.7684 - val_loss: 1327.0477
Epoch 3/20
25/25 [==============================] - 250s 10s/step - loss: 367.2954 - val_loss: 1157.3783
Epoch 4/20
25/25 [==============================] - 255s 10s/step - loss: 292.5784 - val_loss: 1021.7389
Epoch 5/20
25/25 [==============================] - 255s 10s/step - loss: 226.7444 - val_loss: 917.2713
Epoch 6/20
25/25 [==============================] - 262s 10s/step - loss: 177.8259 - val_loss: 985.6646
Epoch 7/20
25/25 [==============================] - 269s 11s/step - loss: 130.9453 - val_loss: 1120.3550


In [300]:
# 테스트셋 MSE 추출
test_loss = model.evaluate([x_0045_test, x_0145_test, x_0245_test, scaled_climate_test], y_test)
test_loss

8/8 [==============================] - 8s 927ms/step - loss: 276.2029


276.20294189453125

In [301]:
# RMSE 계산
np.sqrt(276)

16.61324772583615

In [310]:
# 예측
y_pred = model.predict([x_0045_test, x_0145_test, x_0245_test, scaled_climate_test])

# 예측 결과 확인
print(f'예측값: \n {y_pred[:3]}')
print(f'실제값: \n {y_test[:3]}')

8/8 [==============================] - 7s 807ms/step
예측값: 
 [[66.281044 51.76091  41.7831  ]
 [16.920408 19.871403 21.667334]
 [45.242527 40.65161  34.993305]]
실제값: 
 [[[59.5125    ]
  [95.        ]
  [55.75714286]]

 [[ 3.7875    ]
  [ 9.2125    ]
  [11.9875    ]]

 [[33.7125    ]
  [27.55      ]
  [36.5375    ]]]


In [305]:
x_climate_train.shape

(770, 6, 2220)

In [308]:
x_climate_train.reshape(x_climate_train.shape[0], -1).shape

(770, 13320)

## 이미지 제외하고 LSTM만 돌려보고 비교

In [311]:
# 이미지를 제외했기때문에, 기상, 미세먼지 슬라이딩 윈도우 생성 함수 사용
xx_climate, yy = create_sliding_window(climate_data, seoul_pm)

In [312]:
xx_climate.shape

(1204, 6, 2220)

In [313]:
yy.shape

(1204, 3, 1)

In [314]:
# 데이터 분할 (훈련, 검증, 테스트셋)
xx_climate_train, xx_climate_test, yy_train, yy_test = train_test_split(xx_climate, yy, test_size=0.2, random_state=42)
xx_climate_train, xx_climate_val, yy_train, yy_val = train_test_split(xx_climate_train, yy_train, test_size=0.2, random_state=42)

print(f'train shapes: {xx_climate_train.shape}, {yy_train.shape}')
print(f'validation shapes: {xx_climate_val.shape}, {yy_val.shape}')
print(f'test shapes: {xx_climate_test.shape}, {yy_test.shape}')

train shapes: (770, 6, 2220), (770, 3, 1)
validation shapes: (193, 6, 2220), (193, 3, 1)
test shapes: (241, 6, 2220), (241, 3, 1)


In [316]:
ss = StandardScaler()

In [317]:
xx_climate_train_reshaped = xx_climate_train.reshape(xx_climate_train.shape[0], -1)
xx_climate_val_reshaped = xx_climate_val.reshape(xx_climate_val.shape[0], -1)
xx_climate_test_reshaped = xx_climate_test.reshape(xx_climate_test.shape[0], -1)

xx_climate_train_scaled_reshaped = ss.fit_transform(xx_climate_train_reshaped)
xx_climate_val_scaled_reshaped = ss.transform(xx_climate_val_reshaped)
xx_climate_test_scaled_reshaped = ss.transform(xx_climate_test_reshaped)

xx_scaled_climate_train = xx_climate_train_scaled_reshaped.reshape(xx_climate_train.shape)
xx_scaled_climate_val = xx_climate_val_scaled_reshaped.reshape(xx_climate_val.shape)
xx_scaled_climate_test = xx_climate_test_scaled_reshaped.reshape(xx_climate_test.shape)

In [329]:
# 기상 데이터 입력
input_weather = Input(shape=(6, 2220))

# 첫 번째 LSTM 층
x = LSTM(128, activation='relu', return_sequences=True)(input_weather)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# 두 번째 LSTM 층
x = LSTM(128, activation='relu', return_sequences=True)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# 세 번째 LSTM 층
x = LSTM(128, activation='relu', return_sequences=False)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Fully Connected (FC) 층
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(3, activation='linear')(x)  # 미래 3일 예측

# 모델 생성 및 컴파일
lstm_model = Model(inputs=input_weather, outputs=output)
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 요약
lstm_model.summary()

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_37 (InputLayer)       [(None, 6, 2220)]         0         
                                                                 
 lstm_14 (LSTM)              (None, 6, 128)            1202688   
                                                                 
 batch_normalization_1 (Bat  (None, 6, 128)            512       
 chNormalization)                                                
                                                                 
 dropout (Dropout)           (None, 6, 128)            0         
                                                                 
 lstm_15 (LSTM)              (None, 6, 128)            131584    
                                                                 
 batch_normalization_2 (Bat  (None, 6, 128)            512       
 chNormalization)                                         

In [330]:
# 콜백 설정 (모델 체크포인트, 조기 종료 등)
checkpoint_cb = ModelCheckpoint("./best_LSTM_model.h5", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=8, restore_best_weights=True)

In [331]:
# 모델 훈련
history = lstm_model.fit(xx_scaled_climate_train, yy_train, epochs=100, batch_size=32,
                    validation_data=(xx_scaled_climate_val, yy_val), 
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
25/25 [==============================] - 3s 42ms/step - loss: 1398.7672 - val_loss: 2363.0044
Epoch 2/100
25/25 [==============================] - 1s 32ms/step - loss: 545.4167 - val_loss: 1572.0477
Epoch 3/100
25/25 [==============================] - 1s 32ms/step - loss: 480.5036 - val_loss: 1434.5448
Epoch 4/100
25/25 [==============================] - 1s 32ms/step - loss: 402.0983 - val_loss: 1410.6389
Epoch 5/100
25/25 [==============================] - 1s 31ms/step - loss: 384.3793 - val_loss: 1410.3712
Epoch 6/100
25/25 [==============================] - 1s 31ms/step - loss: 383.9137 - val_loss: 891.4938
Epoch 7/100
25/25 [==============================] - 1s 30ms/step - loss: 378.3284 - val_loss: 972.1300
Epoch 8/100
25/25 [==============================] - 1s 30ms/step - loss: 389.0844 - val_loss: 1359.0769
Epoch 9/100
25/25 [==============================] - 1s 30ms/step - loss: 366.2758 - val_loss: 1116.4764
Epoch 10/100
25/25 [==============================] - 1s

In [332]:
test_loss = lstm_model.evaluate([xx_scaled_climate_test], yy_test)
test_loss

8/8 [==============================] - 0s 11ms/step - loss: 457.7531


457.7531433105469